In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.decomposition import PCA, FastICA

In [2]:
x_train = pd.read_csv('train.csv', index_col='ID')
x_test = pd.read_csv('test.csv', index_col='ID')
y_train = x_train['y']
x_train.drop('y', axis=1, inplace=True)

In [3]:
le = LabelEncoder()
def str_to_int(df, col_name):
    if df[col_name].dtype != 'int64':
        le.fit(df[col_name].values)
        df[col_name] = le.transform(df[col_name])

for col_name in x_train.columns:
    str_to_int(x_train, col_name)
    str_to_int(x_test, col_name)

In [4]:
def save_file(name, df):
    df.to_csv('data/'+name+'.csv', index=False)

### Feature selection and generation

In [5]:
selector = SelectKBest(k=75, score_func=f_regression).fit(x_train, y_train)
x_train_sel = selector.transform(x_train)
x_test_sel = selector.transform(x_test)

/home/iamtodor/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:279: RuntimeWarning: invalid value encountered in true_divide
  corr /= row_norms(X.T)
/home/iamtodor/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/iamtodor/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/iamtodor/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [6]:
x_train['ID'] = x_train.index
x_test['ID'] = x_test.index
y_train['ID'] = y_train.index

In [7]:
x_train_sel_df = pd.DataFrame(x_train_sel)
x_test_sel_df = pd.DataFrame(x_test_sel)

In [8]:
n_comp = 10

# PCA
pca = PCA(n_components=n_comp, random_state=42)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(x_train)
ica2_results_test = ica.transform(x_test)

x_train_pca = pd.DataFrame(x_train_sel)
x_test_pca = pd.DataFrame(x_test_sel)

# Append decomposition components to datasets
for i in range(1, n_comp+1):
    x_train_pca['pca_' + str(i)] = pca2_results_train[:,i-1]
    x_test_pca['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    x_train_pca['ica_' + str(i)] = ica2_results_train[:,i-1]
    x_test_pca['ica_' + str(i)] = ica2_results_test[:, i-1]

In [9]:
save_file(str('x_train'), x_train)
save_file(str('x_test'), x_test)
save_file(str('y_train'), y_train)

save_file(str('x_train_sel'), x_train_sel_df)
save_file(str('x_test_sel'), x_test_sel_df)

save_file(str('x_train_pca'), x_train_pca)
save_file(str('x_test_pca'), x_test_pca)